# Deeplearning computation
### 1. Layers and Blocks

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.0242, -0.1950, -0.1941,  0.0628, -0.0397, -0.0054,  0.2011,  0.1221,
         -0.0935, -0.2975],
        [ 0.0496, -0.2916, -0.0864, -0.0426, -0.0730, -0.0218,  0.2579,  0.0508,
         -0.1628, -0.2782]], grad_fn=<AddmmBackward>)

In [2]:
class MLP(nn.Module):
    def __init__(self):
        # Call the constructor of the `MLP` parent class `Module` to perform the necessary initialization.
        # other function arguments can also be specified during class instantiation, such as the model parameters
        super().__init__()
        self.hidden = nn.Linear(20, 256) 
        self.out = nn.Linear(256, 10) 

    # Define the forward propagation
    # required model output based on the input `X`
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [3]:
net = MLP()
net(X)

tensor([[ 0.0699,  0.1001,  0.2061,  0.3083,  0.1851, -0.1136, -0.0931, -0.0112,
          0.1837,  0.2294],
        [ 0.0458,  0.1339,  0.0996,  0.3237,  0.1989, -0.1045, -0.0070,  0.0321,
          0.1740,  0.1794]], grad_fn=<AddmmBackward>)

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # Here, `module` is an instance of a `Module` subclass. We save it
            # in the member variable `_modules` of the `Module` class, and its
            # type is OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict guarantees that members will be traversed in the order
        # they were added
        for block in self._modules.values():
            X = block(X)
        return X

In [5]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0629,  0.0844, -0.2088, -0.2166,  0.1139,  0.1790, -0.1627,  0.2149,
         -0.0859, -0.0054],
        [-0.0653,  0.0852, -0.1416, -0.0915,  0.1601,  0.2436, -0.2000,  0.2560,
         -0.0263, -0.0032]], grad_fn=<AddmmBackward>)

### 2. Parameter Management

In [6]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.1365],
        [0.1754]], grad_fn=<AddmmBackward>)

In [7]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.3489,  0.1564, -0.2279, -0.0319, -0.3372, -0.1410, -0.3041, -0.1298]])), ('bias', tensor([0.2592]))])


In [8]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4),
                         nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # Nested here
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.3830],
        [0.3830]], grad_fn=<AddmmBackward>)

In [9]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [10]:
rgnet[0][1][0].bias.data

tensor([ 0.1514,  0.4772, -0.3789, -0.1574, -0.2709,  0.1434,  0.2604,  0.1770])

In [11]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0056, -0.0058,  0.0093,  0.0013]), tensor(0.))

In [12]:
def my_init(m):
    if type(m) == nn.Linear:
        print(
            "Init",
            *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000, -0.0000, -6.6474,  8.8518],
        [ 7.7771, -6.5521,  6.6729, -0.0000]], grad_fn=<SliceBackward>)

In [14]:
# We need to give the shared layer a name so that we can refer to its parameters
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared, nn.ReLU(), nn.Linear(8, 1))
net(X)

# Check whether the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100

# Make sure that they are actually the same object rather than just having thesame value
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


### 3. Deferred Initialization

In [15]:
import tensorflow as tf

net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),])

In [16]:
[net.layers[i].get_weights() for i in range(len(net.layers))]

[[], []]

In [17]:
X = tf.random.uniform((2, 20))
net(X)
[w.shape for w in net.get_weights()]

[(20, 256), (256,), (256, 10), (10,)]

### 4. Custom layers

In [18]:
import torch
from torch import nn
from torch.nn import functional as F

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

In [19]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [20]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [21]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(-1.1176e-08, grad_fn=<MeanBackward0>)

In [22]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [23]:
dense = MyLinear(5, 3)
dense.weight

Parameter containing:
tensor([[-0.5551, -1.0702,  1.0787],
        [ 0.4654, -0.2897,  0.4270],
        [ 1.2058, -0.2476,  0.0780],
        [ 0.2457,  1.3174, -0.6553],
        [ 0.7457,  0.6414,  0.4263]], requires_grad=True)

In [24]:
dense(torch.rand(2, 5))

tensor([[2.7336, 2.0793, 0.5800],
        [1.5111, 0.4548, 1.3087]])

In [25]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.],
        [0.]])

### 5. File I/O

In [26]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

In [27]:
x2 = torch.load("x-file")
x2

tensor([0, 1, 2, 3])

In [28]:
y = torch.zeros(4)
torch.save([x, y], 'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [29]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [30]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [31]:
torch.save(net.state_dict(), 'mlp.params')

In [32]:
clone = MLP()
clone.load_state_dict(torch.load("mlp.params"))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [33]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

### 6. GPU

In [34]:
!nvidia-smi

Sat May 29 20:55:15 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 457.30       Driver Version: 457.30       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 1660   WDDM  | 00000000:01:00.0  On |                  N/A |
| 60%   57C    P0    56W / 130W |   3794MiB /  6144MiB |     42%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------